In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from os import path
from tqdm import tqdm
from datetime import datetime

from tensorflow import keras
from tensorflow.keras import layers as l

## Preprocess Data

- We decided to remove rows with null values (given the high abundance of training examples) and to remove rows which included a stray value one day in New York as per https://github.com/CSSEGISandData/COVID-19/issues/3103.
- We assigned the 14-day covid history as separate predictors, as well as a few demographic indicators
- We standardized the data given high variance between the ranges of different features
- We used one-hot encoding for categorical predictors

In [2]:
def num_to_date(i):
    if i>9:
        return str(i)
    return "0" + str(i)

augmented_df = pd.read_csv("../processed_data/combined.csv", index_col=0)

# expand index to multi-index
augmented_df = augmented_df.set_index(["state", "fips"], append=True)

### Drop Null Values

In [3]:
# drop null values
augmented_df = augmented_df.dropna(subset=([f"{k}_before" for k in range(14)] + ['all_poverty', 'pct_impoverished']))

# remove data with spike from NY county https://github.com/CSSEGISandData/COVID-19/issues/3103
augmented_df = augmented_df.drop(augmented_df.loc[[("08-31", "New York", 36061)]+[(f"09-{num_to_date(i)}", "New York", 36061) for i in range(1,15)]].index)

### One-hot encoding

In [4]:
cat = ['state_code', 'c4', 'c6', 'protest_size']
for pred in cat:
    one_hot_encoding = pd.get_dummies(augmented_df[pred], prefix=pred)
    augmented_df = augmented_df.join(one_hot_encoding).drop(columns=[pred])

### Train-Test Split

We did not employ a random train-test split because of the correlations between adjacent examples in the dataset given nature of time-series data. Thust, we split our data into train-test sets at a given date, and then shuffled each set to ensure a more evenly distributed training set.

In [5]:
X = augmented_df.drop(columns=['0_before', 'male', 'female', 'county'])
y = augmented_df['0_before']

test_size = 0.1
split_idx = int(X.shape[0] * (1-test_size))

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

X_train, y_train = shuffle(X_train, y_train, random_state=209)
X_test, y_test = shuffle(X_test, y_test, random_state=209)

### Scale non-categorical features

In [6]:
non_categorical = ['median_age', 'population',
       'female_percentage', 'lat', 'long', 'life_expectancy', 'mortality_risk',
       'all_poverty', 'sq_miles', 'pct_none', 'pct_hs', 'pct_bachelors',
       'median_household_income', 'pct_black', 'pct_asian', 'pct_hispanic',
       'pct_non_hispanic_white', 'pct_not_proficient_in_english', 'pct_rural',
       'pct_impoverished', 'pop_density', 'r_voteshare',
       'stringency', '13_before', '12_before', '11_before',
       '10_before', '9_before', '8_before', '7_before', '6_before', '5_before',
       '4_before', '3_before', '2_before', '1_before']

scaler = StandardScaler().fit(X_train[non_categorical])

# only scale non-categorical variables
X_train_scaled = X_train.copy()
X_train_scaled_noncat = scaler.transform(X_train[non_categorical])
for (i, predictor) in enumerate(non_categorical):
    X_train_scaled[predictor] = X_train_scaled_noncat[:, i]

# only scale non-categorical variables
X_test_scaled = X_test.copy()
X_test_scaled_noncat = scaler.transform(X_test[non_categorical])
for (i, predictor) in enumerate(non_categorical):
    X_test_scaled[predictor] = X_test_scaled_noncat[:, i]

## Prediction



In [7]:
# Divide each example into static and time-series data

# reshape sequences for RNN cells
seq_cols = [f"{k}_before" for k in range(1, 14)] + [f"protest_size_{k}.0" for k in range(-1, 6)]
X_train_seq = X_train_scaled[seq_cols].values.reshape(-1, 1, 20)
print(X_train_seq.shape)

# slice off demographic data
X_train_demo = X_train_scaled.drop(columns=seq_cols)
print(X_train_demo.shape)

(348947, 1, 20)
(348947, 74)


#### Define network architecture

In [8]:
# input layer for sequence- shape is batch_size/1/window_size
seq_in = keras.Input(shape=(1, X_train_seq.shape[2],))

# input layer for demographic data
demo_in = keras.Input(shape=(X_train_demo.shape[1],))

# network for sequences
h1_seq = l.LSTM(32)(seq_in)

# network for demographics with regularization
h1_demo = l.Dense(32, activation='relu')(demo_in)
h1_demo = l.Dropout(0.1)(h1_demo)

# concat
x = l.concatenate([h1_seq, h1_demo])

# dense on top of concatenated layer
x = l.Dense(16, activation='relu')(x)
x = l.Dropout(0.1)(x)

out = l.Dense(1)(x)

#### Compile and summarize model

In [9]:
%load_ext tensorboard

model = keras.Model(
    inputs=[seq_in, demo_in],
    outputs=[out],
)

# use early stopping and save best model to reload it
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='../logs/fit/'+datetime.now().strftime("%Y%m%d-%H%M%S"))
early_stopping_callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
checkpoint_callback = keras.callbacks.ModelCheckpoint('optimum.h5', monitor="val_loss", save_best_only=True)

model.compile(loss='mse')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 74)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 20)]      0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           2400        input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 32)           6784        input_1[0][0]                    
______________________________________________________________________________________________

#### Fit

In [10]:
%tensorboard --logdir ../logs/fit

model.fit(x=[X_train_seq, X_train_demo], y=y_train, epochs=50, validation_split=0.2, callbacks=[tensorboard_callback, early_stopping_callback, checkpoint_callback])

model = keras.models.load_model('optimum.h5')

Train on 279157 samples, validate on 69790 samples
Epoch 1/50
279157/279157 [==============================] - 80s 285us/sample - loss: 4649.4405 - val_loss: 2467.7191
Epoch 2/50
279157/279157 [==============================] - 60s 214us/sample - loss: 4055.1133 - val_loss: 2210.6255
Epoch 3/50
279157/279157 [==============================] - 62s 221us/sample - loss: 3761.2363 - val_loss: 2039.9898
Epoch 4/50
279157/279157 [==============================] - 61s 220us/sample - loss: 3540.0115 - val_loss: 1830.2597
Epoch 5/50
279157/279157 [==============================] - 63s 226us/sample - loss: 3421.1701 - val_loss: 1822.8075
Epoch 6/50
279157/279157 [==============================] - 78s 280us/sample - loss: 3315.4603 - val_loss: 1684.2710
Epoch 7/50
279157/279157 [==============================] - 67s 240us/sample - loss: 3321.9550 - val_loss: 1659.6846
Epoch 8/50
279157/279157 [==============================] - 81s 291us/sample - loss: 3234.3143 - val_loss: 1659.9584
Epoch 9/50
27

#### Evaluation

In [11]:
X_test_seq = X_test_scaled[seq_cols].values.reshape(-1, 1, 20)

X_test_demo = X_test_scaled.drop(columns=seq_cols)

print(f"Train MSE: {model.evaluate(x=[X_train_seq, X_train_demo], y=y_train)}")
print(f"Test MSE: {model.evaluate(x=[X_test_seq, X_test_demo], y=y_test)}")

348947/348947 [==============================] - 35s 100us/sample - loss: 2403.5419- loss:
Train MSE: 2403.5418688863315
38772/38772 [==============================] - 4s 97us/sample - loss: 6571.3510
Test MSE: 6571.350996094002


#### Error Analysis

In [19]:
train_preds = model.predict(x=[X_train_seq, X_train_demo])

print("10 Examples with largest MSEs from training set")
mses = (train_preds[:, 0] - y_train) ** 2
print("5-day case history:")
print(X_train.iloc[mses.argsort()[-10:][::-1]][["1_before", "2_before", "3_before", "4_before", "5_before", "6_before", "7_before"]])
print("\nActual case number:")
print(y_train.iloc[mses.argsort()[-10:][::-1]])
print("\nPredicted case number")
print(train_preds[mses.argsort()[-10:][::-1]])

10 Examples with largest MSEs from training set
5-day case history:
                      1_before  2_before  3_before  4_before  5_before  \
date  state    fips                                                      
09-21 Texas    48201     553.0     819.0     817.0     857.0     435.0   
04-18 New York 36061    4206.0    4844.0    7837.0    3702.0    3555.0   
04-15 New York 36061    3702.0    3555.0    4900.0    5924.0    5356.0   
07-16 Texas    48029       0.0     854.0     565.0    1046.0       0.0   
08-16 Texas    48113     764.0     921.0     750.0     322.0     328.0   
04-11 New York 36061    5356.0    5225.0    4927.0    4695.0    4630.0   
04-09 New York 36061    4927.0    4695.0    4630.0    4245.0    6147.0   
09-23 Texas    48029    3196.0     102.0    2581.0     173.0     162.0   
04-10 New York 36061    5225.0    4927.0    4695.0    4630.0    4245.0   
04-25 New York 36061    4618.0   -1442.0    3107.0    2955.0    2535.0   

                      6_before  7_before  


In [20]:
test_preds = model.predict(x=[X_test_seq, X_test_demo])

print("10 Examples with largest MSEs from training set")
mses = (test_preds[:, 0] - y_test) ** 2
print("5-day case history:")
print(X_test.iloc[mses.argsort()[-10:][::-1]][["1_before", "2_before", "3_before", "4_before", "5_before", "6_before", "7_before"]])
print("\nActual case number:")
print(y_test.iloc[mses.argsort()[-10:][::-1]])
print("\nPredicted case number")
print(test_preds[mses.argsort()[-10:][::-1]])

10 Examples with largest MSEs from training set
5-day case history:
                          1_before  2_before  3_before  4_before  5_before  \
date  state        fips                                                      
11-25 Rhode Island 44007       0.0       0.0       0.0       0.0       0.0   
11-18 Rhode Island 44007       0.0       0.0       0.0       0.0       0.0   
11-30 Texas        48439    1305.0       0.0       0.0       0.0    1302.0   
11-27 Florida      12086       0.0    2120.0    1852.0    1499.0    1746.0   
11-11 Rhode Island 44007       0.0       0.0       0.0       0.0       0.0   
12-03 Texas        48113    1640.0    1179.0    1702.0    2303.0     982.0   
11-15 Florida      12086    1187.0    1876.0    1205.0     718.0     394.0   
11-28 Texas        48201     108.0     596.0    1859.0    2052.0    1060.0   
11-29 Texas        48113     982.0       0.0       0.0    1368.0    1716.0   
11-26 Florida      12086    2120.0    1852.0    1499.0    1746.0    1940.0